# Performance Measuaring using XVal

## Preprocessing

### Loading Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sklearn

In [2]:
SUB_FEATS_FILE_PATH = os.path.join('..', 'features', 'sub_incv1_feats.csv')
SUB_BEST_TECHNIQUES = os.path.join('..', 'results', 'sub_votes_summary.csv')

In [3]:
feats_df = pd.read_csv(SUB_FEATS_FILE_PATH, index_col=0)
feats_df.head(3)

,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
2405479__traffic_light__0.9999939.jpg,0.235278,0.093458,0.096435,0.431311,0.061168,0.201984,0.105483,0.098843,0.309048,0.096289,...,0.071269,0.080100,0.356105,0.096008,0.136125,0.407917,0.707833,0.787253,0.097295,0.010121
2392818__park_bench__0.99999.jpg,0.552391,0.528561,0.226194,1.417185,0.005555,0.519028,0.700005,0.099091,0.123433,0.164651,...,1.730410,0.148950,0.000000,0.065658,0.586209,0.083003,0.054312,0.944901,0.130074,0.047629
2411665__zebra__0.99998856.jpg,0.395213,1.130813,0.120035,1.675753,0.145812,0.117050,0.253090,0.129668,1.047250,0.137647,...,0.000423,0.009721,0.026590,0.914146,0.275013,0.020816,0.503529,0.015693,0.036215,0.005369


In [4]:
techniques_df = pd.read_csv(SUB_BEST_TECHNIQUES, dtype='object', index_col=0)
techniques_df.head(3)

,ig,lime,xrai,anchor,best
2405479__traffic_light__0.9999939.jpg,6,5,3,0,ig
2392818__park_bench__0.99999.jpg,7,2,2,2,ig
2411665__zebra__0.99998856.jpg,5,2,4,2,ig


#### Generate Image-to-Technique mapping

In [120]:
def gen_name_technique_tuples(x):
    return [x.name, x['best']]

In [122]:
foo = techniques_df.apply(gen_name_technique_tuples, axis=1)
foo.values[:3] # Values of Series obj

array([list(['2405479__traffic_light__0.9999939.jpg', 'ig']),
       list(['2392818__park_bench__0.99999.jpg', 'ig']),
       list(['2411665__zebra__0.99998856.jpg', 'ig'])], dtype=object)

In [123]:
name_tech_map = {name: tech for name, tech in foo.values}

### Renaming data

In [5]:
X = feats_df.values
y = techniques_df.values[:, -1]

In [147]:
X_names = feats_df.index.values

In [152]:
print(X.shape, X_names.shape, y.shape)

(126, 1024) (126,) (126,)


#### Deleting instances

Stratified Subsampling cannot be performed onto the dataset because only one instance is best explained with ANCHOR. Due to the very small importance of that instance in the dataset, we will continue without that instance (i.e. we will find tha instance and remove it form the dataset)

In [20]:
anchor_idxs = np.argwhere(y == 'anchor')[0]
anchor_idxs

array([103], dtype=int64)

In [21]:
y[anchor_idxs]

array(['anchor'], dtype=object)

In [22]:
X = np.delete(X, anchor_idxs, axis=0)
X.shape

(126, 1024)

In [151]:
X_names = np.delete(X_names, anchor_idxs, axis=0)
X_names.shape

(126,)

In [23]:
y = np.delete(y, anchor_idxs, axis=0)
y.shape

(126,)

### Test-Train Splitting

We are splitting a dataset that does not include an ANCHOR image. Take that in account.

In [28]:
from sklearn.model_selection import StratifiedShuffleSplit as SSS

In [29]:
splitter = SSS(n_splits=5, test_size=0.2, random_state=42)

In [30]:
splits = splitter.split(X, y)

In [31]:
splits = list(splits)

Five different splits, each split being the combination of training indices and test indices

In [32]:
splits[0]

(array([ 28,  41,  33,  65,  26, 109,  63, 116,  36,  54,  84,   5,  13,
         22,  72,  56, 117,  19, 112, 120,  46,  88,  30,  74,  34,  69,
         24,  92, 102,  40,  11,  81, 124,   9, 104,  35,  70,  98,   3,
        113,  91,  16,  86,  93,  42, 123, 122,  53,  76,  79,  90,  17,
        100, 118, 115,  97,  75,  55,  64,  85,  48,  68,  67, 114,  44,
         61,  95,  89,  47,  43, 101,   6, 119, 111,   7,  57,  87,  49,
         18,  25,  62,   8,  82,  39,   0,  27,  78,  15, 125,  38,  96,
         58, 103,  50,  31,  12, 106,  45,   4,  10], dtype=int64),
 array([ 71,  37,  66, 108,   2,  29,  14, 105,  51,  60,  80,  20,  23,
         94,  77,  73, 121,  52,  32,  21,  83,   1, 107, 110,  59,  99],
       dtype=int64))

In [48]:
len(splits[0][0])

100

## Clustering

### Finding Clustering Parameters

The min. no. of clustered instances should be about 70% of the dataset. The training split of the dataset contains 100 instances, so we need to cluster at least 70 instances. 

In [49]:
# Arrays of arrays [m, epsilon, # split]
clusterable_params = []

In [50]:
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score

In [51]:
def fit_dbscan(data, min_samples, eps_values, 
               min_no_clusters=2, max_no_clusters=np.inf,
               min_clust_instances=70, max_clust_instances=np.inf, metric='euclidean'):
    # Code
    scores, clusters, instances = [], [], []
    for m in min_samples:
        row_scores, row_clusters, row_instances = [], [], []
        for e in eps_values:
            db = DBSCAN(min_samples=m, eps=e, metric=metric).fit(data)
            # Get only non anomalous instances
            non_a = db.labels_ != -1 # [False, ..., False] if all are outliers
            # Calculate conditions
            n_clusters = len(np.unique(db.labels_[non_a])) # 0 if all are outliers
            n_instances = len(db.labels_[non_a]) # 0 if all are outliers
            # Apply conditions (why does it output NaN and not None?)
            valid_n_clusters = n_clusters >= min_no_clusters and n_clusters <= max_no_clusters
            valid_n_cl_instances = n_instances >= min_clust_instances and n_instances <= max_clust_instances
            if (valid_n_clusters and valid_n_cl_instances):
                score = silhouette_score(data[non_a], db.labels_[non_a], metric=metric)
            else:
                score = None
            # Store results
            row_scores.append(score)
            row_clusters.append(n_clusters)
            row_instances.append(n_instances)
        # Store row results
        scores.append(row_scores)
        clusters.append(row_clusters)
        instances.append(row_instances)
    # Prepare and return values
    ms_axis = pd.Index(min_samples, name='Min_samples')
    eps_axis = pd.Index(eps_values, name='Epsilon')
    df_scores = pd.DataFrame(scores, index=ms_axis, columns=eps_axis)
    df_clusters = pd.DataFrame(clusters, index=ms_axis, columns=eps_axis)
    df_instances = pd.DataFrame(instances, index=ms_axis, columns=eps_axis)
    return df_scores, df_clusters, df_instances

In [52]:
def print_results(m, eps, scores_df, instances_df, clusters_df):
    score = scores_df.loc[m][eps]
    instances = instances_df.loc[m][eps]
    clusters = clusters_df.loc[m][eps]
    print(f'DBSCAN using parameters m={m} and eps={eps} yields the next clustering results:')
    print()
    print(f'- Sil. score: {score}')
    print(f'- {instances} clustered instances into {clusters} clusters')
    print(f'- Avg. of {instances/clusters} instances per cluster')

#### Split #0

In [53]:
dfs, dfc, dfi = fit_dbscan(X[splits[0][0]], range(2, 9), range(10, 17))
dfs

Epsilon,10,11,12,13,14,15,16
Min_samples,,,,,,,
2,None,None,0.391798,0.376625,0.307936,0.202339,0.163839
3,None,None,NaN,0.396278,0.325839,0.256537,NaN
4,None,None,NaN,0.396278,0.325839,0.256537,NaN
5,None,None,NaN,NaN,0.317820,0.256537,NaN
6,None,None,NaN,NaN,0.314682,NaN,NaN
7,None,None,NaN,NaN,0.314682,NaN,NaN
8,None,None,NaN,NaN,NaN,NaN,NaN


In [60]:
print_results(3, 13, dfs, dfi, dfc)

DBSCAN using parameters m=3 and eps=13 yields the next clustering results:

- Sil. score: 0.3962778790867661
- 74 clustered instances into 7 clusters
- Avg. of 10.571428571428571 instances per cluster


In [61]:
clusterable_params.append([3, 13, 0])

#### Split #1

In [63]:
dfs, dfc, dfi = fit_dbscan(X[splits[1][0]], range(2, 8), range(10, 18))
dfs

Epsilon,10,11,12,13,14,15,16,17
Min_samples,,,,,,,,
2,None,None,0.452305,0.436532,0.309734,0.264342,0.224398,0.346657
3,None,None,0.457988,0.435077,0.304582,0.257638,0.361834,0.346657
4,None,None,NaN,NaN,0.313422,0.270622,NaN,NaN
5,None,None,NaN,NaN,NaN,0.285752,NaN,NaN
6,None,None,NaN,NaN,NaN,0.285752,NaN,NaN
7,None,None,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
print_results(3, 12, dfs, dfi, dfc)

DBSCAN using parameters m=3 and eps=12 yields the next clustering results:

- Sil. score: 0.4579880021486161
- 73 clustered instances into 7 clusters
- Avg. of 10.428571428571429 instances per cluster


In [65]:
clusterable_params.append([3, 12, 1])

#### Split #2

In [67]:
dfs, dfc, dfi = fit_dbscan(X[splits[2][0]], range(2, 6), range(10, 18))
dfs

Epsilon,10,11,12,13,14,15,16,17
Min_samples,,,,,,,,
2,None,None,0.408135,0.398702,0.349018,0.214244,0.201258,0.337595
3,None,None,NaN,NaN,0.362647,0.258577,0.357435,0.337595
4,None,None,NaN,NaN,0.361382,0.253341,NaN,NaN
5,None,None,NaN,NaN,0.354800,0.258334,NaN,NaN


In [74]:
print_results(2, 12, dfs, dfi, dfc)

DBSCAN using parameters m=2 and eps=12 yields the next clustering results:

- Sil. score: 0.40813547009093765
- 78 clustered instances into 13 clusters
- Avg. of 6.0 instances per cluster


In [75]:
clusterable_params.append([2, 12, 2])

#### Split #3

In [78]:
dfs, dfc, dfi = fit_dbscan(X[splits[3][0]], range(2, 6), range(10, 18))
dfs

Epsilon,10,11,12,13,14,15,16,17
Min_samples,,,,,,,,
2,None,None,0.420049,0.379916,0.252146,0.283625,0.347257,0.335138
3,None,None,NaN,0.407446,0.286289,0.283625,0.347257,0.335138
4,None,None,NaN,0.421617,0.282102,0.279475,NaN,NaN
5,None,None,NaN,NaN,0.271381,0.296204,NaN,NaN


In [79]:
print_results(4, 13, dfs, dfi, dfc)

DBSCAN using parameters m=4 and eps=13 yields the next clustering results:

- Sil. score: 0.4216169632381272
- 71 clustered instances into 6 clusters
- Avg. of 11.833333333333334 instances per cluster


In [80]:
clusterable_params.append([4, 13, 3])

#### Split #4

In [82]:
dfs, dfc, dfi = fit_dbscan(X[splits[4][0]], range(2, 6), range(10, 18))
dfs

Epsilon,10,11,12,13,14,15,16,17
Min_samples,,,,,,,,
2,None,None,0.436515,0.426429,0.301934,0.226202,0.238915,0.292161
3,None,None,NaN,0.430633,0.297537,0.325589,0.312530,0.292161
4,None,None,NaN,NaN,0.312725,0.325589,0.312530,0.292161
5,None,None,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
print_results(2, 12, dfs, dfi, dfc)

DBSCAN using parameters m=2 and eps=12 yields the next clustering results:

- Sil. score: 0.4365150087588584
- 78 clustered instances into 12 clusters
- Avg. of 6.5 instances per cluster


In [84]:
clusterable_params.append([2, 12, 4])

### Clustering Analysis

After detecting the parameters used to generate well defined clusters, we will use those params. to cluster the features again and calculate the most frequent technique of every cluster on every clustering result.

In [86]:
cl_params_df = pd.DataFrame(clusterable_params, columns=['m', 'eps', 'split'])
cl_params_df

,m,eps,split
0,3,13,0
1,3,12,1
2,2,12,2
3,4,13,3
4,2,12,4


#### Clustering Results

In [111]:
def get_indiv_clustering_results(param_set):
    '''Returns a dictionary mapping the name of an image
    with the cluster it belongs'''
    # Prepare parameters
    split_no = param_set[2]
    data = feats_df.iloc[splits[split_no][0]]
    img_names = data.index.values
    instances = data.values
    # Perform clustering
    dbscan = DBSCAN(min_samples=param_set[0], eps=param_set[1])
    dbscan = dbscan.fit(instances)
    # Generate {img_name : label} mapping
    name_label_map = {name: label for name, label in zip(img_names, dbscan.labels_)}
    return name_label_map

def get_global_clustering_results(params):
    '''Returns a dictionary mapping the index of every param set
    in 'params' arg. with the clustering results generated with that param. set'''
    results = {}
    for i, param_set in enumerate(params):
        results[i] = get_indiv_clustering_results(param_set)
    return results

In [112]:
cl_results = get_global_clustering_results(cl_params_df.values)

In [117]:
cl_results

{0: {'2385461__zebra__0.99998415.jpg': 0,
  '2409472__umbrella__0.9999845.jpg': 1,
  '2406209__brown_bear__0.999985.jpg': -1,
  '2409637__four-poster__0.99999464.jpg': -1,
  '2377620__zebra__0.9999882.jpg': 0,
  '2413521__parking_meter__0.99999976.jpg': 2,
  '2406887__ski__0.99999785.jpg': 3,
  '2415567__zebra__0.9999902.jpg': 0,
  '2391408__traffic_light__0.99999654.jpg': 4,
  '2391918__steam_locomotive__0.99999845.jpg': -1,
  '2392324__traffic_light__0.9999964.jpg': 5,
  '2378170__zebra__0.9999902.jpg': 0,
  '2409063__zebra__0.999997.jpg': 0,
  '2379489__parking_meter__0.9999989.jpg': 2,
  '2407951__zebra__0.9999975.jpg': 0,
  '2384512__zebra__0.99999726.jpg': 0,
  '2415648__broccoli__0.9999943.jpg': -1,
  '2414335__refrigerator__0.9999889.jpg': -1,
  '2414384__ski__0.9999949.jpg': 3,
  '2416228__parking_meter__0.9999914.jpg': 2,
  '2404229__zebra__0.99998367.jpg': 0,
  '2398771__traffic_light__0.99999.jpg': 4,
  '2412027__school_bus__0.99998724.jpg': -1,
  '2404583__umbrella__0.9999

#### Clustering Proportions

In [126]:
def get_indiv_clustering_proportions(name_label_map, name_best_tech_map=name_tech_map):
    # Clustering proportions
    cl_props = {}
    for name, cl_label in name_label_map.items():
        # Add cluster label to clustering proportions if it hasn't been added yet
        if (cl_label not in cl_props.keys()): cl_props[cl_label] = {}
        # If best technique is in current cluster
        technique = name_best_tech_map[name]
        if (technique in cl_props[cl_label].keys()):
            cl_props[cl_label][technique] += 1 # add one...
        else:
            cl_props[cl_label][technique] = 1 # else, create with one
    return cl_props

def get_global_clustering_proportions(cl_results):
    '''Returns a dictionary mapping the index of every param set
    used for the clustering results with the best technique proportions
    of every individual cluster'''
    results = {}
    for param_set_idx, name_label_map in cl_results.items():
        results[param_set_idx] = get_indiv_clustering_proportions(name_label_map)
    return results

In [127]:
cl_props = get_global_clustering_proportions(cl_results)

In [128]:
cl_props

{0: {0: {'ig': 25, 'xrai': 11, 'lime': 2, 'anchor': 1},
  1: {'ig': 3, 'lime': 2},
  -1: {'ig': 14, 'xrai': 5, 'lime': 5},
  2: {'xrai': 6, 'ig': 6},
  3: {'ig': 2, 'lime': 2},
  4: {'ig': 2, 'lime': 2, 'xrai': 1},
  5: {'lime': 2, 'xrai': 1, 'ig': 3},
  6: {'ig': 5}},
 1: {0: {'ig': 3},
  -1: {'lime': 8, 'ig': 11, 'xrai': 5},
  1: {'xrai': 14, 'ig': 29, 'lime': 2, 'anchor': 1},
  2: {'ig': 8, 'xrai': 5},
  6: {'lime': 2, 'ig': 1},
  3: {'ig': 4, 'lime': 1},
  4: {'ig': 2, 'lime': 1},
  5: {'ig': 2, 'xrai': 1}},
 2: {0: {'ig': 7, 'xrai': 3},
  1: {'lime': 2, 'ig': 24, 'xrai': 15, 'anchor': 1},
  2: {'ig': 2},
  -1: {'ig': 10, 'lime': 5, 'xrai': 5},
  3: {'ig': 2, 'lime': 2},
  4: {'ig': 2},
  5: {'ig': 2},
  6: {'ig': 1, 'lime': 1, 'xrai': 1},
  7: {'ig': 4},
  8: {'xrai': 1, 'ig': 1},
  9: {'lime': 2},
  10: {'lime': 1, 'ig': 1},
  11: {'ig': 3},
  12: {'ig': 1, 'lime': 1}},
 3: {0: {'ig': 27, 'xrai': 14, 'lime': 2, 'anchor': 1},
  -1: {'ig': 16, 'lime': 8, 'xrai': 5},
  5: {'lime': 2

### Generating Labels

Once the technique proportions of each cluster have been calculated, we'll associate each image with the majority technique present in the cluster it belongs to. These associations will be used as training labels in the future.

In [137]:
def get_maj_technique_per_cluster(cluster_votes):
    higher_vote_count = 0
    prefered_technique = None
    for technique, vote_count in cluster_votes.items():
        if vote_count > higher_vote_count:
            higher_vote_count = vote_count
            prefered_technique = technique
    return prefered_technique

def get_maj_techniques(cl_prop):
    cl_techniques = {}
    for cl_label, cl_votes in cl_prop.items():
        cl_techniques[cl_label] = get_maj_technique_per_cluster(cl_votes)
    return cl_techniques
    
def gen_labels_per_clustering(cl_result, cl_prop):
    # Get most frequent techniques for each cluster
    cl_techniques = get_maj_techniques(cl_prop)
    # Propagate maj tehnique to all intances inside each cluster
    new_labels = {img_name:cl_techniques[cl_label] for img_name,cl_label in cl_result.items()}
    return new_labels

def gen_labels(cl_results, cl_props):
    labels = {}
    for i in range(len(cl_results)):
        labels[i] = gen_labels_per_clustering(cl_results[i], cl_props[i])
    return labels

In [138]:
labels = gen_labels(cl_results, cl_props)

In [139]:
labels

{0: {'2385461__zebra__0.99998415.jpg': 'ig',
  '2409472__umbrella__0.9999845.jpg': 'ig',
  '2406209__brown_bear__0.999985.jpg': 'ig',
  '2409637__four-poster__0.99999464.jpg': 'ig',
  '2377620__zebra__0.9999882.jpg': 'ig',
  '2413521__parking_meter__0.99999976.jpg': 'xrai',
  '2406887__ski__0.99999785.jpg': 'ig',
  '2415567__zebra__0.9999902.jpg': 'ig',
  '2391408__traffic_light__0.99999654.jpg': 'ig',
  '2391918__steam_locomotive__0.99999845.jpg': 'ig',
  '2392324__traffic_light__0.9999964.jpg': 'ig',
  '2378170__zebra__0.9999902.jpg': 'ig',
  '2409063__zebra__0.999997.jpg': 'ig',
  '2379489__parking_meter__0.9999989.jpg': 'xrai',
  '2407951__zebra__0.9999975.jpg': 'ig',
  '2384512__zebra__0.99999726.jpg': 'ig',
  '2415648__broccoli__0.9999943.jpg': 'ig',
  '2414335__refrigerator__0.9999889.jpg': 'ig',
  '2414384__ski__0.9999949.jpg': 'ig',
  '2416228__parking_meter__0.9999914.jpg': 'xrai',
  '2404229__zebra__0.99998367.jpg': 'ig',
  '2398771__traffic_light__0.99999.jpg': 'ig',
  '241

## Evaluation

With avaliable labels, we can predict interpretation techniques for new images, and also evaluate the performance of those prediction using metrics such as accuracy.

In [158]:
list(labels[0].values())

['ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'xrai',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'xrai',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'xrai',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'xrai',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'xrai',
 'xrai',
 'xrai',
 'ig',
 'xrai',
 'ig',
 'ig',
 'ig',
 'ig',
 'xrai',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'xrai',
 'xrai',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'ig',
 'xrai']

In [154]:
y

array(['ig', 'ig', 'ig', 'ig', 'ig', 'ig', 'ig', 'ig', 'ig', 'ig', 'ig',
       'ig', 'ig', 'ig', 'ig', 'ig', 'ig', 'ig', 'ig', 'ig', 'ig', 'ig',
       'ig', 'ig', 'ig', 'ig', 'ig', 'ig', 'ig', 'ig', 'ig', 'ig', 'ig',
       'ig', 'ig', 'ig', 'ig', 'ig', 'ig', 'ig', 'ig', 'ig', 'ig', 'ig',
       'ig', 'ig', 'ig', 'ig', 'ig', 'ig', 'ig', 'ig', 'ig', 'ig', 'ig',
       'ig', 'ig', 'ig', 'ig', 'ig', 'ig', 'ig', 'ig', 'ig', 'ig', 'ig',
       'ig', 'ig', 'ig', 'ig', 'ig', 'ig', 'ig', 'ig', 'ig', 'ig', 'lime',
       'lime', 'lime', 'lime', 'xrai', 'lime', 'xrai', 'xrai', 'lime',
       'lime', 'lime', 'lime', 'lime', 'xrai', 'xrai', 'xrai', 'xrai',
       'lime', 'xrai', 'lime', 'xrai', 'xrai', 'xrai', 'lime', 'xrai',
       'lime', 'xrai', 'xrai', 'xrai', 'lime', 'xrai', 'lime', 'xrai',
       'xrai', 'xrai', 'lime', 'xrai', 'xrai', 'xrai', 'xrai', 'xrai',
       'xrai', 'xrai', 'xrai', 'xrai', 'xrai', 'lime', 'xrai', 'xrai',
       'xrai'], dtype=object)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder

def evaluate_clustering(labels, split_no):
    cl_metrics = {}
    # Fetch train/test data
    train_indices, test_indices = splits[split_no][0], splits[split_no][1]
    train_data, test_data = X[train_indices], X[test_indices]
    train_techniques, test_techniques = labels.values(), y[test_indices]
    train_img_names, test_img_names = X_names[train_indices], X_names[test_indices]
    # Encode labels
    encoder = LabelEncoder.fit(train_techniques)
    train_labels = encoder.transform(train_techniques)
    test_labels = encoder.transform(test_techniques)
    
    # Train KNN model
    knn = KNeighborsClassifier(n_neighbors=3)
    # Generate predictions
    predictions = {}
    for test_img in test_data
        predictions[test_img_name] = find_pred_technique(test_img)
    # Evaluate predictions
    cl_metrics['accuracy'] = 0.8
    return cl_metrics

def evaluate_clusterings(labels_set):
    global_metrics = {}
    for i in range(labels_set):
        global_metrics[i] = evaluate_clustering(labels_set[i], i)
    return global_metrics